# Using C Shared Library with Python

The advantage of Python is that it is **flexible and easy** to program. The time it takes to setup a new calulation is therefore short. But for certain types of calculations Python (and any other interpreted language) can be **very slow**. It is particularly iterations over large arrays that is difficult to do efficiently.

Such calculations may be implemented in a compiled language such as C or Fortran. In Python it is relatively easy to call out to libraries with compiled C or Fortran code. 

But before we go ahead and work on optimizing anything, it is always worthwhile to ask.... 

In [1]:
import seuif97
%timeit seuif97.pt(15,535,4)

The slowest run took 18644.46 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 7.17 µs per loop


In [2]:
from iapws.iapws97 import IAPWS97
%timeit IAPWS97(P=16.10,T=535.10).h

1000 loops, best of 3: 602 µs per loop


The **slowest** run took xxx.xx times **longer** than the **fastest**. This could mean that an **intermediate result** is being **cached.**

* Algorithm of the High-speed IAPWS-IF97 Library: 

   * 王培红,贾俊颖,程懋华. 水和水蒸汽热力性质IAPWS_IF97公式的通用计算模型[J]. 动力工程. 2001 21(6)：1564-1567(EI)


## C/C++ Programing with GCC(for Windows)

### 1. GCC: GNU Compiler Collection

#### 1.1 A Brief History and Introduction to GCC

* The original GNU C Compiler (GCC) is developed by Richard Stallman, the founder of the GNU Project. Richard Stallman founded the GNU project in 1984 to create a complete Unix-like operating system as free software, to promote freedom and cooperation among computer users and programmers.

  GCC, formerly for "GNU C Compiler", has grown over times to support many languages such as C++, Objective-C, Java, Fortran and Ada. It is now referred to as "GNU Compiler Collection". 
  The mother site for GCC is **http://gcc.gnu.org/**.

  GCC is a key component of "GNU Toolchain", for developing applications, as well as operating systems. 

  GCC is portable and run in many operating platforms. GCC (and GNU Toolchain) is currently available on all Unixes. They are also ported to Windows by MinGW and Cygwin. GCC is also a cross-compiler, for producing executables on different platform.

#### 1.2  Installing MinGW GCC

GCC (GNU Toolchain) is included in all Unixes. For Windows, you could either install **MinGW GCC** or **Cygwin GCC**.

* MinGW-w64 GCC

  MinGW (short for "Minimalist GNU for Windows"), is a minimalist (i.e., small but fewer features compared with cygwin) development environment for native Microsoft Windows applications, in particular:
   * 1.A port of the GNU Compiler Collection (GCC), including C, C++, ADA and Fortran compilers;
   * 2.GNU Binutils for Windows (assembler, linker, archive manager).
   * 3.MSYS (short for "Minimal SYStem"), is a bash Shell command line interpreter.


* To install MinGW-w64:
  
  * 1.Goto MinGW mother site at https://sourceforge.net/projects/mingw-w64/files/?source=navbar 

      Downloads x86_64-6.3.0-release-win32-seh-rt_v5-rev1.7z 

      https://sourceforge.net/projects/mingw-w64/files/Toolchains%20targetting%20Win64/Personal%20Builds/mingw-builds/6.3.0/threads-win32/seh/x86_64-6.3.0-release-win32-seh-rt_v5-rev1.7z
    
  * 2.unzip the ziped MinGW-w64 to C:\mingw64

  * 3.Setup **C:\mingw6\bin** to PATH  
      ```bash
      echo off
      set PATH=C:\mingw64\bin;%PATH%
      echo %PATH%
      ```

   * 4.Verify the GCC installation by listing the version of gcc, g++ and gdb: 
      ```bash
      > gcc --version
      > g++ --version
      > gdb --version
      ```


In [ ]:
!echo off
!set PATH=C:\mingw64\bin;%PATH%
!echo %PATH%

In [1]:
!gcc --version
!g++ --version
!gdb --version

gcc (x86_64-win32-seh-rev1, Built by MinGW-W64 project) 6.3.0
Copyright (C) 2016 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

g++ (x86_64-win32-seh-rev1, Built by MinGW-W64 project) 6.3.0
Copyright (C) 2016 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

GNU gdb (GDB) 7.11.1
Copyright (C) 2016 Free Software Foundation, Inc.
License GPLv3+: GNU GPL version 3 or later <http://gnu.org/licenses/gpl.html>
This is free software: you are free to change and redistribute it.
There is NO WARRANTY, to the extent permitted by law.  Type "show copying"
and "show warranty" for details.
This GDB was configured as "x86_64-w64-mingw32".
Type "show configuration" for configuration details.
For bug reporting instructions, please see:
<h

### 1.3  Getting Started

The GNU C and C++ compiler are gcc and g++, respectively.

**Compile/Link a Simple C Program - hello.c**

Below is the Hello-world C program hello.c:


In [2]:
%%file hello.c

#include <stdio.h>
 
int main() {
    printf("C says Hello, world!\n");
    return 0;
}

Writing hello.c


To compile the hello.c:

In [3]:
!gcc -o hello.exe hello.c

To run the program:

In [4]:
!hello

C says Hello, world!


**Compile/Link a Simple C++ Program - hello.cpp**

Below is the Hello-world C++ program hello.cpp:


In [ ]:
%%file hello.cpp

#include <iostream>
using namespace std;
 
int main() {
   cout << "C++ Hello, world!" << endl;
   return 0;
}

You need to use g++ to compile C++ program, as follows. We use the -o option to specify the output file name.


In [ ]:
!g++ -o hello.exe hello.cpp

To run the program:

In [ ]:
!hello

## ctypes

ctypes is a Python library for calling out to C code. We manually need to load the library and set properties such as the functions return and argument types. On the otherhand we do not need to touch the C code at all. 

In [ ]:
%%file funs.c

#include <stdio.h>

void hello(int n);
double dprod(double *x, int n);
void dcumsum(double *a, double *b, int n);

void hello(int n)
{
    for (int i = 0; i < n; i++)
    {
        printf("C says Hello world!\n");
    }
}

double dprod(double *x, int n)
{
    double y = 1.0;
    for (int i = 0; i < n; i++)
    {
        y *= x[i];
    }
    return y;
}

void dcumsum(double *a, double *b, int n)
{
    b[0] = a[0];
    for (int i = 1; i < n; i++)
    {
        b[i] = a[i] + b[i-1];
    }
}

Compile the C file into a shared library:

In [ ]:
!gcc -c -O3 -Wall -fPIC -o funs.o funs.c
!gcc -o libfuns.dll -shared funs.o

The result is a compiled shared library `libfuns.dll`

Now we need to write wrapper functions to access the C library: 
To load the library we use the **ctypes** package, which included in the Python standard library (with extensions from numpy for passing arrays to C). Then we manually set the types of the argument and return values (no automatic code inspection here!). 

In [ ]:
%%file funs.py

from ctypes import c_int, c_double,c_void_p
import numpy as np

_libfuns = np.ctypeslib.load_library('libfuns', '.')

_libfuns.hello.argtypes = [c_int]
_libfuns.hello.restype  =  c_void_p

_libfuns.dprod.argtypes = [np.ctypeslib.ndpointer(dtype=np.float), c_int]
_libfuns.dprod.restype  = c_double

_libfuns.dcumsum.argtypes = [np.ctypeslib.ndpointer(dtype=np.float), np.ctypeslib.ndpointer(dtype=np.float),c_int]
_libfuns.dcumsum.restype  = c_void_p

def hello(n):
    return _libfuns.hello(int(n))

def dprod(x, n=None):
    if n is None:
        n = len(x)
    x = np.asarray(x, dtype=np.float)
    return _libfuns.dprod(x, int(n))

def dcumsum(a, n):
    a = np.asarray(a, dtype=np.float)
    b = np.empty(len(a), dtype=np.float)
    _libfuns.dcumsum(a, b, int(n))
    return b

In [ ]:
%%file run_hello_c.py

import funs
funs.hello(3)

In [ ]:
!python run_hello_c.py

### Product function:

In [ ]:
import funs
funs.dprod([1,2,3,4,5]) 

### Cummulative sum:

In [ ]:
import funs
a=[1,2,3,4,5]
result = funs.dcumsum(a, len(a)) 
print(result)

### Further reading

* http://docs.python.org/3/library/ctypes.html
* http://www.scipy.org/Cookbook/Ctypes